# Aggregation

## 1. 종합

### 1.1 단일열 기준 종합

#### groupby 

In [10]:
import pandas as pd
import numpy as np

In [11]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


### 1.2 복수열 기준 종합

In [12]:
flights.groupby(['AIRLINE','WEEKDAY'])['CANCELLED'].agg('sum').head()
#에어라인이 있고 기준에 대한 종합(통합)을 내는 함수

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
Name: CANCELLED, dtype: int64

In [16]:
flights.groupby(['AIRLINE','WEEKDAY'])['CANCELLED','DIVERTED'].agg(['sum','mean']).head()
#그룹바이 잡고 통계하면된단다/


CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786

In [17]:
group_cols = ['ORG_AIR','DEST_AIR']
#리스트에서 중첩리스트를 쓴다
agg_dict = {'CANCELLED':['sum','mean','size'], 'AIR_TIME':['mean','var']}
flights.groupby(group_cols).agg(agg_dict).head()
#aggregation종합적으로 보여준다!


CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

## MUltiIndex 제거

In [20]:
air_info = flights.groupby(group_cols).agg(agg_dict).head()
air_info

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

In [22]:
# get_level_values
level0 = air_info.columns.get_level_values(0)
level0

Index(['CANCELLED', 'CANCELLED', 'CANCELLED', 'AIR_TIME', 'AIR_TIME'], dtype='object')

In [23]:
level1 = air_info.columns.get_level_values(1)
level1

Index(['sum', 'mean', 'size', 'mean', 'var'], dtype='object')

In [26]:
air_info.columns = level0 + '_'+level1
air_info.head(3)
#각 값을 더해줘서 이름을 만들고 값을 추출한다

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643

In [27]:
#인덱스를 다시 셋팅하자@
air_info.reset_index().head()

,ORG_AIR,DEST_AIR,CANCELLED_sum,CANCELLED_mean,CANCELLED_size,AIR_TIME_mean,AIR_TIME_var
0,ATL,ABE,0,0.0,31,96.387097,45.778495
1,ATL,ABQ,0,0.0,16,170.500000,87.866667
2,ATL,ABY,0,0.0,19,28.578947,6.590643
3,ATL,ACY,0,0.0,6,91.333333,11.466667
4,ATL,AEX,0,0.0,40,78.725000,47.332692


In [29]:
air_info = flights.groupby(group_cols, as_index = False).agg(agg_dict)
air_info.head(3)


ORG_AIR DEST_AIR CANCELLED              AIR_TIME           
                         sum mean size        mean        var
0     ATL      ABE         0  0.0   31   96.387097  45.778495
1     ATL      ABQ         0  0.0   16  170.500000  87.866667
2     ATL      ABY         0  0.0   19   28.578947   6.590643

In [31]:
air_info = flights.groupby(group_cols).agg(agg_dict)
air_info.head(3)
#as_index를 사용하는 방법을 29와 30에서 차이점을 보고 사용해보자

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643

## 2. 사용자 정의 함수 사용

In [34]:
def pct_between(s):
    return s.between(-100, 60).mean()

In [37]:
flights.groupby(['AIRLINE'])['DEP_DELAY', 'ARR_DELAY'].agg(pct_between)
#평균값을 알려주었따
#정의 함수를 만들어서 써보자
#

,DEP_DELAY,ARR_DELAY
AIRLINE,,
AA,0.916854,0.915281
AS,0.972656,0.971354
B6,0.915285,0.907919
DL,0.959249,0.958211
EV,0.917890,0.912940
F9,0.913440,0.900532
HA,0.982143,0.973214
MQ,0.897148,0.889369
NK,0.868734,0.852902


In [38]:
flights.groupby(['AIRLINE'])['DEP_DELAY', 'ARR_DELAY'].agg(['mean',pct_between])

#agg는 각각의 값 렬을 만들어서 넣어주었다.


DEP_DELAY              ARR_DELAY            
              mean pct_between       mean pct_between
AIRLINE                                              
AA       11.274057    0.916854   5.542661    0.915281
AS        1.808594    0.972656  -0.833333    0.971354
B6       14.287823    0.915285   8.692593    0.907919
DL        7.242450    0.959249   0.339691    0.958211
EV        9.092625    0.917890   7.034580    0.912940
F9       14.310398    0.913440  13.630651    0.900532
HA        2.571429    0.982143   4.972973    0.973214
MQ       11.094323    0.897148   6.860591    0.889369
NK       19.514401    0.868734  18.436070    0.852902
OO        9.662279    0.911809   7.593463    0.905282
UA       16.210485    0.902207   7.765755    0.908881
US        4.361355    0.954799   1.681105    0.954799
VX        8.918946    0.938570   5.348884    0.940584
WN       12.639366    0.929912   6.397353    0.930981

In [39]:
def pct_between(s, low, high):
    return s.between(low, high).mean()

In [41]:
flights.groupby(['AIRLINE'])['DEP_DELAY', 'ARR_DELAY'].agg(pct_between, -10, 10)
#

,DEP_DELAY,ARR_DELAY
AIRLINE,,
AA,0.749213,0.398090
AS,0.751302,0.510417
B6,0.681400,0.366483
DL,0.805867,0.461089
EV,0.753329,0.495050
F9,0.665907,0.447229
HA,0.794643,0.491071
MQ,0.705848,0.397292
NK,0.607520,0.422164


## Exercies
대학 데이터셋을 사용해 소수 인종이 다수인 주 찾기

In [43]:
college = pd.read_csv('data/college.csv', index_col='INSTNM')
college.head()

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [45]:
college.STABBR.value_counts().sum()

7535

In [46]:
grouped =college.groupby('STABBR')

In [48]:
def check_m(df, threshold):
    m_pct = 1-df['UGDS_WHITE'] 
    total_m = (df['UGDS'] * m_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_m_pct = total_m / total_ugds
    return total_m_pct > threshold

In [51]:
c_filter = grouped.filter(check_m, threshold = 0.5)
c_filter.head()

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,...,0.0373,0.0,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,...,0.0241,0.0,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,...,0.0400,0.0,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,...,0.0000,0.0,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,1.0,...,0.0000,0.0,0.0000,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacySuppressed
